In [14]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
import geopy
from geopy.distance import vincenty
from pandas.plotting import scatter_matrix
import seaborn as sns

import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

from IPython.display import Image
from IPython.core.display import HTML
from IPython.display import HTML

from pylab import rcParams
import itertools
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, pacf, adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX

from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings("ignore")

init_notebook_mode(connected=True)

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" 
value="toggle on/off the raw code"></form>''')

Below is a summary of my analysis, for full deep dive, please go to: http://localhost:8888/notebooks/Bike_Share_WX_Impact/Bike%20Share%20and%20Weather.ipynb

# Does weather change bike riding habits in San Francisco?

<p style="font-size:16px">From my analysis on bike share date in SF and weather I found that weather indeed impacts bike riding behaviour. I came to this conclusion by analyzing bike and weather data with [EDA](./full_analysis.html#eda) and correlation plots. I also built time series models to understand if weather is a good predictor for daily bike share rides. Following, I am going to present you my findings. </p>

### Average number of bike share rides is at highest when its mild weather

<p style="font-size:16px">People are more likely to use bike-share when the weather is good. Average number of bike-share rides is 3 times higher compared to all other conditions. We can conclude that people use bike-share more often when the weather is good.</p>

In [11]:
bike_wx = pd.read_csv('bike_wx.csv')
days = list(bike_wx.groupby('date').nunique().count())[0]
rain_ct = list(bike_wx[bike_wx.precip>0].groupby('wx_feel').bike_id.count()/days)
no_rain_ct= list(bike_wx[bike_wx.precip<=0].groupby('wx_feel').bike_id.count()/days)

# Defining a color palette
light_blue = 'rgb(142, 212, 229)'
pink = 'rgb(231, 84, 128)'
green = 'rgb(199, 204, 118)'
grey = 'rgb(169,169,169)'
yellow = 'rgb(248, 222, 126)'

trace1 = go.Bar(
    x=['Cold & 🌧', 'Good & 🌧', 'Hot & 🌧'],
    y=rain_ct, marker = dict(color=light_blue),
    name='Rain')
trace2 = go.Bar(
    x=['Cold & 🌤', 'Good & 🌤', 'Hot & 🌤'],
    y=no_rain_ct,
    name='No Rain', marker = dict(color=yellow))

data = [trace1, trace2]
layout = go.Layout(
    barmode='stack', title = 'Daily average number of bike share rides by weather and precipation', xaxis=dict(title='Weather condition'),
    yaxis=dict(
        title='Average number of bike share rides'),)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename = "Weather")

<font size="2">$^*$Cold is below 53 F, Good is beween 53 F and 73 F and Hot is above 73 F.  </font>

### Bike rides duration increases with better weather

<font size="3">The weather also impacts bike share ride duration. On sunny days with no rain the rides are almost 20.8% longer and the avergae duration increases with imrpoving weather conditions.</font>

In [21]:
increase = (np.mean(np.array(no_rain_avg_dur)) - np.mean(np.array(rain_avg_dur)))/np.mean(np.array(rain_avg_dur))
rain_avg_dur = list(bike_wx[bike_wx.precip>0].groupby('wx_feel').duration_sec.mean()/60)
no_rain_avg_dur = list(bike_wx[bike_wx.precip<=0].groupby('wx_feel').duration_sec.mean()/60)
trace1 = go.Bar(
    x=['Cold & 🌧', 'Good & 🌧', 'Hot & 🌧'],
    y=rain_avg_dur, marker = dict(color=light_blue),
    name='Rain')
trace2 = go.Bar(
    x=['Cold & 🌤', 'Good & 🌤', 'Hot & 🌤'],
    y=no_rain_avg_dur,
    name='No Rain', marker = dict(color=yellow))

data = [trace1, trace2]
layout = go.Layout(
    barmode='stack', title = 'Average duration of a bike share ride (in minutes) by weather and precipation', xaxis=dict(title='Weather condition'),
    yaxis=dict(
        title='Average duration of a bike share ride'),)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename = "Weather")

### Share of Customer riding increases when it's hot

<font size="3">I found out that weather also impacts behavior of different customer segments. The proportion of Customers increases by 10.8 percentage points vs. Subscribers with improving weather condition from cold to hot.</font>

In [24]:
bt = pd.read_csv('bike_sharing_weather_daily.csv')

In [26]:
# Good
cust_g = bt[bt.wx_feel == 'good'].count_customer.sum() / bt[bt.wx_feel == 'good'].count_shares.sum()
sub_g = bt[bt.wx_feel == 'good'].count_subscriber.sum() / bt[bt.wx_feel == 'good'].count_shares.sum()
# Cold
cust_c = bt[bt.wx_feel == 'cold'].count_customer.sum() / bt[bt.wx_feel == 'cold'].count_shares.sum()
sub_c = bt[bt.wx_feel == 'cold'].count_subscriber.sum() / bt[bt.wx_feel == 'cold'].count_shares.sum()
# Hot
cust_h = bt[bt.wx_feel == 'hot'].count_customer.sum() / bt[bt.wx_feel == 'hot'].count_shares.sum()
sub_h = bt[bt.wx_feel == 'hot'].count_subscriber.sum() / bt[bt.wx_feel == 'hot'].count_shares.sum()

trace1 = go.Bar(
    x=['Cold', 'Good', 'Hot'],
    y=[cust_c, cust_g, cust_h], marker = dict(color=grey),
    name='Customer')
trace2 = go.Bar(
    x=['Cold', 'Good', 'Hot'],
    y=[sub_c, sub_g, sub_h],
    name='Subscriber', marker = dict(color=pink))

data = [trace1, trace2]
layout = go.Layout(
    barmode='stack', title = 'Comparing customers vs. subscribers behaviour by weather', 
    xaxis=dict(title='Weather condition'),
    yaxis=dict(
        title='% of total bike share rides'),)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename = "Bike Shares")

See more analysis and insights about weather impact on Customer vs. Subscriber behaviour in my Deep Dive Analysis here: http://localhost:8888/notebooks/Bike_Share_WX_Impact/Bike%20Share%20and%20Weather.ipynb

In [ ]:
<font size="3"></font>